In [ ]:
!pip install plotly
!pip install pandas

In [ ]:
rez = original_table_rag.loc[(original_table_rag['version'] == 2) & (original_table_rag['score'] < 10)]

In [ ]:
rez['has_paris'] = rez.apply(lambda row: 'Paris' in row['context'], axis=1)

In [ ]:
original_table_rag.loc[(original_table_rag['version'] == 2) & (original_table_rag['score'] < 10), 'context'].value_counts()

In [ ]:
import json
import pandas as pd
import plotly.express as px

with open('output/results_gpt_rag.json', 'r') as file:
    results = json.load(file)

original_table_rag = pd.DataFrame(results)
table_rag = original_table_rag.pivot_table(index='depth_percent', columns='context_length', values='score') / 10
mask = (table_rag >= 0.3)
table_rag = table_rag.where(mask, 0)

table_long_context = pd.read_csv('original_results/gpt4.csv', index_col=0) / 10

In [ ]:
table_long_context['depth_percent'] = table_long_context.index

In [ ]:
table_long_context = table_long_context.melt(id_vars='depth_percent', var_name='context_length', value_name='score')
table_long_context['context_length'] = table_long_context['context_length'].astype(int)

In [ ]:
grouped_long_context = table_long_context.groupby([pd.cut(table_long_context['depth_percent'], 5), pd.cut(table_long_context['context_length'], 4)])['score'].mean().unstack()
grouped_long_context

In [ ]:
table_rag['depth_percent'] = table_rag.index.astype(int)
table_rag = table_rag.melt(id_vars='depth_percent', var_name='context_length', value_name='score')
table_rag['context_length'] = table_rag['context_length'].astype(int)
grouped_rag = table_rag.groupby([pd.cut(table_rag['depth_percent'], 5), pd.cut(table_rag['context_length'], 4)])['score'].mean().unstack()
grouped_rag

In [ ]:
series_long_context = table_long_context.groupby(pd.cut(table_long_context['context_length'], 5))['score'].mean()
series_long_context = pd.DataFrame(series_long_context).rename(columns={'score': 'average_score'}).T

In [ ]:
series_rag = table_rag.groupby(pd.cut(table_rag['context_length'], 5))['score'].mean()
series_rag = pd.DataFrame(series_rag).rename(columns={'score': 'average_score'}).T

In [ ]:
series_long_context.columns = [str(col) for col in series_long_context.columns]
series_rag.columns = [str(col) for col in series_rag.columns]

In [ ]:
comb = pd.concat([series_long_context, series_rag], axis=0).T
comb.columns = ['long_context', 'rag']

In [ ]:
comb = pd.DataFrame(comb.unstack().reset_index())
comb.columns = ['model', 'context_length', 'average_score']

In [ ]:
comb

In [ ]:
fig = px.line(comb, color="model", x='context_length', y='average_score', title='Accuracy of Retrieval - RAG vs Long-Context GPT4', width=600)
fig.update_layout(yaxis_range=[0, 1], yaxis_tickformat = ',.0%')

### Plot as heatmap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import matplotlib
import matplotlib as mpl

def display_table(table):
    fig, ax = plt.subplots()
    im = ax.imshow(table)

    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(table.columns)), labels=table.columns)
    ax.set_yticks(np.arange(len(table.index)), labels=table.index)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
            rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(table.index)):
        for j in range(len(table.columns)):
            text = ax.text(j, i, table.values[i, j],
                        ha="center", va="center", color="w")

    fig.tight_layout()
    plt.show()

display_table(grouped_rag)
display_table(grouped_long_context)

### Plot as graphs

In [ ]:
!pip install plotly

In [ ]:
graph_rag = pd.DataFrame(grouped_rag.stack()).reset_index()
graph_rag['depth_percent'] = graph_rag['depth_percent'].apply(lambda x: x.mid).astype(int)
graph_rag['context_length'] = graph_rag['context_length'].apply(lambda x: x.mid).astype(int)

graph_long_context = pd.DataFrame(grouped_long_context.stack()).reset_index()
graph_long_context['depth_percent'] = graph_long_context['depth_percent'].apply(lambda x: x.mid).astype(int)
graph_long_context['context_length'] = graph_long_context['context_length'].apply(lambda x: x.mid).astype(int)

In [ ]:
import plotly.express as px

px.line(graph_rag, x='depth_percent', color='context_length', y=0, width=600)



In [ ]:
px.line(graph_long_context, x='depth_percent', color='context_length', y=0, width=600)